# A/B Test: Effect of Name Presence at Intake on Adoption Outcomes within 30 Days

## Summary & Recommendation

Having a name corresponds to a directional increase in adoption rate within 30 days, with low operational cost and low risk, although the evidence is not causal.

I recommend a limited rollout of assigning temporary names to unnamed cats, while monitoring adoption rate as the primary metric and length of stay as a guardrail metric.

## Business Question

> **Should the animal center assign names to all unnamed cats to improve adoption outcomes?**

The goal of this analysis is to evaluate whether having a name improves adoption outcomes.

This intervention occurs right after intake, early in the adoption funnel, and affects how cats are presented in listings and communications.

## Hypothesis

A cat's name evokes positive emotions, such as excitement or affection, from potential pet owners. Thus, 
> *adding a name increases adoption rate within 30 days of intake*.

## Metrics

- Primary metric: Adoption rate within 30 days of intake ("30-day adoption rate")
- Guardrail: Length of stay (LOS) among cats adopted within 30 days of intake

Trade-offs considered are operational effort versus expected lift, and adoption speed versus volume.

## Experimental Design & Constraints

- Control group: Cats without a name at intake
- Treatment group: Cats with a name at intake
- Population: Intakes with observed outcomes from similar intake types ("Owner Surrender", "Abandoned", and "Stray")

The analysis compares **naturally occurring groups** (named vs unnamed cats) rather than randomly assigned treatments, so it’s observational, not randomized. Therefore, results are interpreted as **directional evidence, not causal proof**. Noise from seasonality and intake mix is expected.

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

parent_dir = str(Path().resolve().parent)
sys.path.append(parent_dir)

df = pd.read_csv("../data/processed/aac_processed.csv", parse_dates=["datetime_intake"])

In [2]:
# define name presence at intake
df["has_name"] = df["name_intake"].notna()

# filter for intakes with outcomes
df_ab = df[df["has_outcome"]].copy()

# limit intake types to similar intake types
df_ab = df_ab[df_ab["intake_type"].isin(["Owner Surrender", "Abandoned", "Stray"])]

# define outcome window: adopted within 30 days
df_ab["adopted_30d"] = (df_ab["is_adopted"]) & (df_ab["length_of_stay_days"] <= 30)

## Exploratory Checks

**37% (40,694) of named cats and 17% (26,394) of unnamed cats are adopted within 30 days**. The 30-day adoption rate is more than two times higher among named cats than among unnamed cats, with a **17% uplift from name presence**.

In [3]:
df_ab.groupby("has_name").agg(
    adoption_rate_30d=("adopted_30d", "mean"),
    count=("adopted_30d", "size")
).sort_values(by="adoption_rate_30d", ascending=False)

,adoption_rate_30d,count
has_name,,
True,0.368236,40694
False,0.167387,26394


## Statistical Comparison

Named cats show a higher 30-day adoption rate than unnamed cats. **The difference in 30-day adoption rate is statistically significant** under a two-proportion z-test. 

As name assignment is not randomized, this result should be interpreted as directional rather than causal. However, the effect is operationally meaningful and low-cost, low-risk to implement.

In [4]:
from src.ab_test import z_test

# perform a two-proportion z-test
z_stat, p_value, rates = z_test(df_ab)
print(f"Z-stat: {z_stat:.4f}, p-value: {p_value:.4f}")
print(f"30-day adoption rate among named cats: {rates[0]:.2%}.")
print(f"30-day adoption rate among unnamed cats: {rates[1]:.2%}.\n")

Z-stat: 56.0513, p-value: 0.0000
30-day adoption rate among named cats: 36.82%.
30-day adoption rate among unnamed cats: 16.74%.



## Guardrail Check (LOS)

**Among cats adopted within 30 days, named cats stay at the center for 3-4 days longer than unnamed cats. The difference in LOS is statistically significant** under Mann–Whitney U test.

Save for times of overflowing capacity or understaffed personnel at the center, a difference of 3-4 days in LOS will not affect the center if the 30-day adoption rate is significantly higher. 


In [5]:
df_adopted_30d = df_ab[df_ab["adopted_30d"]].copy()
df_adopted_30d.groupby("has_name")["length_of_stay_days"].agg(["mean", "median"]).sort_values(by="mean", ascending=False)

,mean,median
has_name,,
True,12.032766,9.0
False,8.175192,6.0


In [6]:
from src.ab_test import mwu_test

# perform a Mann–Whitney U test
U_statistic, p_value = mwu_test(df_adopted_30d)
print(f"Z-stat: {U_statistic:.4f}, p-value: {p_value:.4f}")

Z-stat: 42219371.5000, p-value: 0.0000


## Results 

**Named cats have a significantly higher 30-day adoption rate**. The difference in LOS is small and not adverse. 

Given the low cost of the implementation, the risk of inaction likely exceeds the risk of action.

## Final Recommendation

Assigning names at intake shows a directional lift in 30-day adoption rates, with no clear evidence of downside risk. The effect size may be modest, but at the scale of the animal center, it adds up.

Given the low cost and minimal operational risk, I **recommend a limited rollout: assign temporary names** to unnamed cats at intake and **monitor 30-day adoption rate as the primary metric, with LOS as a guardrail**. This captures upside while containing any risks and avoids delaying action for a perfect signal.

## Potential Next Step: Randomization

If feasible, the next step would be a **randomized test at intake**, where unnamed cats are randomly assigned to receive a temporary name or not. Randomization would remove intake-related confounding factors and allow for a clearer estimate of the causal effect of naming.

The primary decision metric would remain 30-day adoption rate, with LOS as a guardrail. The goal to evaluate the effect well enough to decide whether naming cats should become a permanent policy.

# Appendix: Adoption Uplift Simulation (For Planning Purposes)

## Purpose

This appendix simulates the potential impact of adding a name under different rollout scenarios. 

As assignment was not randomized, the observed differences between named and unnamed cats should be interpreted as directional rather than causal. To support a real policy decision, I combine this observational evidence with simulations that apply a conservative uplift of 10% to each cat’s baseline adoption probability. This approach shifts the question from *“is the difference statistically significant?”* to *“in practice, does naming meaningfully improve adoption outcomes at scale?”*.

## Simulation

In [7]:
from src.features import add_baseline_adoption_prob

# filter for unnamed cats with an outcome recorded
df_sim = df[(~df["has_name"]) & (df["has_outcome"])].copy()

# add a heuristic baseline adoption probability for simulation
df_sim = add_baseline_adoption_prob(df_sim)

In [8]:
from src.ab_test import simulations, compute_30d_adoptions

ROLL_OUT_RATES = [0.25, 0.50, 0.75, 1.00]  # 0%, 25%, 50%, 75%, and full rollout
N_SIM = 1000 # number of simulation
UPLIFT = 0.10  # assumed absolute uplift from having a name

# perform simulations 
sim_results = simulations(df_sim, ROLL_OUT_RATES, N_SIM, UPLIFT, seed=2026)

# quantify potential impact
sim_results_adoption_counts = compute_30d_adoptions(df, sim_results)
sim_results_adoption_counts

Annual intakes of unnamed cats: 2087.
Baseline 30-day adoption rate among unnamed cats: 16.40%.



,rollout_rate,expected_30d_adoption_rate,std_dev,annual_30d_adoptions,increased_30d_adoptions
0,0.25,0.568511,0.002987,1186.351701,844.120932
1,0.50,0.593519,0.002897,1238.536293,896.305523
2,0.75,0.618561,0.002895,1290.794631,948.563861
3,1.00,0.643643,0.002875,1343.133533,1000.902764


## Results

Based on the simulation, naming cats that arrive without names is expected to produce a directional increase in adoptions within 30 days, with impact scaling proportionally to rollout size. 

**At current intake volumes (~2,100 no-name cats/year), a full rollout assuming a 10% uplift corresponds to approximately 1000 additional 30-day adoptions per year**, while partial rollouts capture a meaningful share of that upside with lower operational effort.

These estimates do not prove causality and should be interpreted as planning inputs rather than forecasts. The results assume a stable intake mix and do not account for post-intake constraints such as shelter resources or staffing. Given the low downside risk and low implementation cost in naming cats, I recommend a limited rollout with monitoring to validate impact before scaling further.